In [7]:
from minicons import scorer
from nltk.tokenize import TweetTokenizer
from torch.utils.data import DataLoader
import csv

In [14]:
word_tokenizer = TweetTokenizer().tokenize

In [15]:
model = "kanishka/smolm-autoreg-bpe-babylm-1e-3"
device = "cpu"

if "gpt2" in model or "pythia" in model or "SmolLM" in model:
    BOS = True
else:
    BOS = False

lm = scorer.IncrementalLMScorer(model, device=device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


model.safetensors:   0%|          | 0.00/391M [00:00<?, ?B/s]

In [8]:
def read_csv_dict(file_path):
    with open(file_path, mode='r') as f:
        reader = csv.DictReader(f)
        data = [row for row in reader]
    return data

In [9]:
stimuli = read_csv_dict("test-sentences.csv")

In [10]:
stimuli

[{'sequence_id': '1', 'sequence': 'The keys to the cabinet are on the table.'},
 {'sequence_id': '2', 'sequence': 'The keys to the cabinet is on the table.'}]

In [11]:
batch_size = 4

stimuli_dl = DataLoader(stimuli, batch_size=batch_size, shuffle=False)

In [16]:
for batch in stimuli_dl:
    word_scores = lm.word_score_tokenized(
        batch['sequence'],
        bos_token=BOS,
        tokenize_function=word_tokenizer,
        surprisal=True,
        base_two=True,
    )